In [1]:
import pickle

import music21
import torch
import torch.nn as nn
import torch.nn.functional as F

In [57]:
file_to_predict = input("Input a single part musicXML file to predict the chord progression for: ")

Input a single part musicXML file to predict the chord progression for: ../../data/test_data/test_inputs/Poster_Melody.musicxml


In [3]:
model_fp = input("Input the file path of the model to use for chord prediction: ")

Input the file path of the model to use for chord prediction: ../../data/models/model10.pt


In [84]:
prediction_file_name = input("Input the name of the file to save the predicted multipart piece to: ")

Input the name of the file to save the predicted multipart piece to: ../../data/test_data/test_outputs/Poster_MelodyWC3.musicxml


In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(device)

cuda:0


In [6]:
use_start_bit = False

In [59]:
bins_per_measure = 16

score = music21.converter.parse(file_to_predict)

step_length = 4 / bins_per_measure

melody = score.parts[0]

melody.show("text")

melody_measures = melody.getElementsByClass(music21.stream.Measure)

test_data = []
for i, mel_measure in enumerate(melody_measures):
    test_measure = []
    if mel_measure.timeSignature is not None:
        step_length = (mel_measure.timeSignature.numerator / mel_measure.timeSignature.denominator
                       * 4 / bins_per_measure)
    melody_elements = [item for item in mel_measure.notesAndRests]
    melody_index = 0

    melody_quintet = []
    for j in range(bins_per_measure):
        offset_timestep = j * step_length

        if (melody_elements[melody_index] is not melody_elements[-1] 
            and melody_elements[melody_index + 1].offset <= offset_timestep):
            melody_index += 1

        melody_item = melody_elements[melody_index]
        
        item_index = melody_item.pitch.pitchClass if melody_item.name != "rest" else 12
        item_vector = [1 if k == item_index else 0 for k in range(13)]
        if use_start_bit:
            item_vector.append(1 if i == 0 else 0)
            
        melody_quintet += item_vector

        if j % 4 == 3:
            test_data.append(melody_quintet)
            melody_quintet = []
    
melody_tensor = torch.tensor(test_data, dtype=torch.float, device=device)

{0.0} <music21.instrument.Instrument 'P1: Melody: Piano'>
{0.0} <music21.stream.Measure 1 offset=0.0>
    {0.0} <music21.layout.SystemLayout>
    {0.0} <music21.clef.TrebleClef>
    {0.0} <music21.key.KeySignature of no sharps or flats>
    {0.0} <music21.meter.TimeSignature 4/4>
    {0.0} <music21.note.Rest rest>
    {1.0} <music21.note.Note E>
    {1.5} <music21.note.Note G>
    {2.0} <music21.note.Note A>
    {2.5} <music21.note.Note G>
    {3.0} <music21.note.Note E>
    {3.5} <music21.note.Note C>
{4.0} <music21.stream.Measure 2 offset=4.0>
    {0.0} <music21.note.Note E->
    {0.5} <music21.note.Note D>
    {1.5} <music21.note.Note D>
    {3.5} <music21.note.Note C>
{8.0} <music21.stream.Measure 3 offset=8.0>
    {0.0} <music21.note.Note A>
    {0.5} <music21.note.Note C>
    {1.5} <music21.note.Note C>
    {3.0} <music21.note.Note A>
    {3.5} <music21.note.Note G>
{12.0} <music21.stream.Measure 4 offset=12.0>
    {0.0} <music21.note.Note E>
{16.0} <music21.stream.Measure 5 offs

In [60]:
print(melody_tensor)
print(melody_tensor.shape)

tensor([[0., 0., 0.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
torch.Size([32, 52])


In [61]:
# class LSTMGenerator(nn.Module):
    
#     def __init__(self):
#         super(LSTMGenerator, self).__init__()
#         self.embedding = nn.Embedding(13, 100)
#         self.lstm = nn.LSTM(input_size=100, hidden_size=256, num_layers=2, batch_first=True)
#         self.fc1 = nn.Linear(256, 12)
        
#     def forward(self, x, hidden_in):
#         x = self.embedding(x)
#         x, h_out = self.lstm(x, hidden_in)
#         x = self.fc1(x)
#         return x, h_out

# model = LSTMGenerator()

In [62]:
# class LSTMGenerator_v2(nn.Module):
    
#     def __init__(self):
#         super(LSTMGenerator_v2, self).__init__()
#         self.lstm = nn.LSTM(input_size=13, 
#                             hidden_size=256, 
#                             num_layers=2, 
#                             batch_first=True, 
#                             bidirectional=True, 
#                             dropout=0.2)
#         self.fc1 = nn.Linear(512, 256)
#         self.fc2 = nn.Linear(256, 12)
        
#     def forward(self, x, hidden_in):
#         x, h_out = self.lstm(x, hidden_in)
#         x = self.fc1(x)
#         x = F.relu(x)
#         x = self.fc2(x)
        
#         return x, h_out

# model = LSTMGenerator_v2()

# model.to(device)

In [63]:
# Define recurrent prediction model
class LSTMGenerator_v5(nn.Module):
    def __init__(self):
        super(LSTMGenerator_v5, self).__init__()
        self.lstm = nn.LSTM(input_size=52, 
                            hidden_size=256, 
                            num_layers=2, 
                            batch_first=True, 
                            bidirectional=True, 
                            dropout=0.2)
        self.fc1 = nn.Linear(512, 256)
        self.fc2 = nn.Linear(256, 12)
        
    def forward(self, x, hidden_in):
        x, h_out = self.lstm(x, hidden_in)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        
        return x, h_out

model = LSTMGenerator_v5()

model.to(device)
print(net)

pytorch_total_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
print(pytorch_total_params)

LSTMGenerator_v5(
  (lstm): LSTM(52, 256, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (fc1): Linear(in_features=512, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=12, bias=True)
)
2346252


In [64]:
model.load_state_dict(torch.load(model_fp))
model.eval()

LSTMGenerator_v5(
  (lstm): LSTM(52, 256, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (fc1): Linear(in_features=512, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=12, bias=True)
)

In [65]:
melody_tensor = melody_tensor.unsqueeze(0)

In [66]:
print(melody_tensor.shape)

torch.Size([1, 32, 52])


In [82]:
hidden_state = (torch.zeros(2 * 2, 1, 256, device=device),
                torch.zeros(2 * 2, 1, 256, device=device))

chord_tensor = (torch.sigmoid(model(melody_tensor, hidden_state)[0]) > 0.5)
first_largest = -1
second_largest = -1
third_largest = -1
chord_tensor.squeeze_()

tensor([[1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
        [1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
        [1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0],
        [1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
        [1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0],
        [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0],
        [1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0],
        [1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0],
        [1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0],
        [1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0],
        [1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
        [1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
        [1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
        [1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
        [1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
        [1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
        [1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
        [1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0],
        [1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0],
        [1, 0, 0, 0, 0, 1, 0, 0, 0

In [83]:
new_part = music21.stream.Part()
new_part.insert(0, music21.instrument.Piano())

time_sig = music21.meter.TimeSignature("4/4")

for measure in torch.chunk(chord_tensor, int(chord_tensor.shape[0] / 4)):
    new_measure = music21.stream.Measure()
    new_measure.insert(0, time_sig)
    pitch_classes = [[i for i, val in enumerate(chord_vector) if val == 1] for chord_vector in measure]
    pitch_class_to_quarterlength = [[pitch_classes[0], (4 * 4 / bins_per_measure)]]
    for i, p_class in enumerate(pitch_classes[1:], 1):
        if p_class == pitch_classes[i - 1]:
            pitch_class_to_quarterlength[-1][1] += (4 * 4 / bins_per_measure)
        else:
            pitch_class_to_quarterlength.append([p_class, (4 * 4 / bins_per_measure)])
            
    for pitch_class_list, quarterlength in pitch_class_to_quarterlength:
        if pitch_class_list != []:
            new_measure.append(music21.chord.Chord(pitch_class_list, quarterLength=quarterlength))
        else:
            new_measure.append(music21.note.Rest(quarterLength=quarterlength))
            
    new_part.append(new_measure)

new_part.show("text")

{0.0} <music21.instrument.Piano 'Piano'>
{0.0} <music21.stream.Measure 0 offset=0.0>
    {0.0} <music21.meter.TimeSignature 4/4>
    {0.0} <music21.chord.Chord C E G>
    {2.0} <music21.chord.Chord C E G B->
    {3.0} <music21.chord.Chord C E G>
{4.0} <music21.stream.Measure 0 offset=4.0>
    {0.0} <music21.meter.TimeSignature 4/4>
    {0.0} <music21.chord.Chord C E G>
    {1.0} <music21.chord.Chord D G B->
    {3.0} <music21.chord.Chord C F A>
{8.0} <music21.stream.Measure 0 offset=8.0>
    {0.0} <music21.meter.TimeSignature 4/4>
    {0.0} <music21.chord.Chord C F A>
    {3.0} <music21.chord.Chord C E G>
{12.0} <music21.stream.Measure 0 offset=12.0>
    {0.0} <music21.meter.TimeSignature 4/4>
    {0.0} <music21.chord.Chord C E G>
{16.0} <music21.stream.Measure 0 offset=16.0>
    {0.0} <music21.meter.TimeSignature 4/4>
    {0.0} <music21.chord.Chord C E G>
    {3.0} <music21.chord.Chord C F A>
{20.0} <music21.stream.Measure 0 offset=20.0>
    {0.0} <music21.meter.TimeSignature 4/4>
   

In [85]:
score.insert(0, new_part)

gex = music21.musicxml.m21ToXml.GeneralObjectExporter(score)
    
out = gex.parse()
musicxml = out.decode('utf-8').strip()
    
with open(prediction_file_name, "w") as outfile:
    outfile.write(musicxml)